In [148]:
%reset -f
import sys
sys.path.append('/home/psimmerl/mds_analysis')

import pathlib

import math
import numpy as np
import numba as nb
from math import ceil, floor

from src.histo_utilities import std_color_list as SCL
# from src.helper_functions import alert, Table, weight_calc

from src import CMS_lumi, tdrstyle
root_style = tdrstyle.setTDRStyle()

import ROOT as rt
from ROOT import RDataFrame
from ROOT import TCanvas, TLatex, TLegend, TLine, TBox
from ROOT import TH1D, TH2D, TGraph, TGraphErrors, TGraphAsymmErrors

# **************** #

CTAU = 1000
# CTAUS = np.round(np.logspace(2, 4, 10), 0)
# CTAUS = np.round(np.logspace(1, 7, 55), 0)
CTAUS = np.round(np.logspace(1, 5, 30), 0)
if CTAU not in CTAUS:
    CTAUS = np.append(CTAUS, CTAU)
    CTAUS = np.sort(CTAUS)

# **************************** #
LOCAL_DIR = '/home/psimmerl/mds_analysis'
OUT_DIR = f'{LOCAL_DIR}/reports/weekly/2024-04-15'

# **** #
MET_CATEGORY = ('lt200', 'low', 'high')[1]
TAG_CATEGORY = ('csccsc', 'cscdt')[1]
CUTSET = ('l1', 'scs', 'ropt', 'roptDNN', 'roptDNN_bkgMC_plusBeamHalo', 'roptDNN_bkgOOTData', 'lopt', 'loptDNN')[4]
OOT = False#True#

SIZE_VAR = TAG_CATEGORY[3:]

FN_MC = f'{LOCAL_DIR}/data/processed/mc_{TAG_CATEGORY}_{CUTSET}_{MET_CATEGORY}_rdf.root'
# FN_MC = f'{LOCAL_DIR}/data/processed/mc_{TAG_CATEGORY}{"OOT" if OOT else ""}_{CUTSET}_{MET_CATEGORY}_rdf.root'
FN_R3 = f'{LOCAL_DIR}/data/processed/r3_{TAG_CATEGORY}{"OOT" if OOT else ""}_{CUTSET}_{MET_CATEGORY}_rdf.root'

# **** #
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# rt.gErrorIgnoreLevel = 1001  # rt.kInfo + 1
# rt.gROOT.SetBatch(True)
PI = rt.TMath.Pi()

# rt.EnableImplicitMT(4)
# print('Enabled ROOT\'s implicit multithreading (sometimes causes a crash)')

gc = []

# **************** #
print(f'{MET_CATEGORY=}')
print(f'{TAG_CATEGORY=}')
print(f'{CUTSET=}')
print(f'{OOT=}')
print('')

print(f'{FN_MC=}')
print(f'{FN_R3=}')
print(f'{OUT_DIR=}')
print('')

# **************** #
rdfs = {
    'mc' : RDataFrame('MuonSystem_flat', FN_MC),
    'r3' : RDataFrame('MuonSystem_flat', FN_R3),
    #
    'mc, No CSC Jet Veto' : RDataFrame('MuonSystem_flat', FN_MC.replace('_rdf','_noCSCJetVeto_rdf')),
    'r3, No CSC Jet Veto' : RDataFrame('MuonSystem_flat', FN_R3.replace('_rdf','_noCSCJetVeto_rdf')),
    #
    'mc, No DT Jet Veto' : RDataFrame('MuonSystem_flat', FN_MC.replace('_rdf','_noDTJetVeto_rdf')),
    'r3, No DT Jet Veto' : RDataFrame('MuonSystem_flat', FN_R3.replace('_rdf','_noDTJetVeto_rdf')),
    #
    'mc, No CSC Muon Veto' : RDataFrame('MuonSystem_flat', FN_MC.replace('_rdf','_noCSCMuonVeto_rdf')),
    'r3, No CSC Muon Veto' : RDataFrame('MuonSystem_flat', FN_R3.replace('_rdf','_noCSCMuonVeto_rdf')),
    #
    'mc, No DT Muon Veto' : RDataFrame('MuonSystem_flat', FN_MC.replace('_rdf','_noDTMuonVeto_rdf')),
    'r3, No DT Muon Veto' : RDataFrame('MuonSystem_flat', FN_R3.replace('_rdf','_noDTMuonVeto_rdf')),
}

print('Events Read:')
for key, rdf in rdfs.items():
    if TAG_CATEGORY == 'csccsc':
        # rdf = rdf.Filter('tag_dR > 1.5')
        rdf = rdf.Alias('tag_size', 'csc1Size')
        rdf = rdf.Alias('tag_ctau', 'csc1CTau')

    if TAG_CATEGORY == 'cscdt':
        rdf = rdf.Filter('tag_dPhi > 0.4')
        rdf = rdf.Alias('tag_size', 'dtSize')
        rdf = rdf.Alias('tag_ctau', 'dtCTau')

    count, weight = rdf.Count().GetValue(), rdf.Sum('weight').GetValue()
    print(f'  {key} = {count:,} ({weight:,.2f})')

    rdfs[key] = rdf


CUT_VERSION='l1'
TAG_CATEGORY='cscdt'
SIZE_VAR='dt'

FN_MC='/home/psimmerl/mds_analysis/data/processed/mc_cscdt_l1_lt200_rdf.root'
FN_R3='/home/psimmerl/mds_analysis/data/processed/r3_cscdtOOT_l1_lt200_rdf.root'
OUT_DIR='/home/psimmerl/mds_analysis/reports/weekly/2024-03-27'

Events Read:
  mc = 3,824 (2,310.04)
  r3 = 5,082 (5,082.00)
